In [1]:
!sudo apt-get install -y python3-mpi4py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  python3-numpy
The following NEW packages will be installed:
  python3-mpi4py
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 465 kB of archives.
After this operation, 1,857 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-mpi4py amd64 3.1.3-1build2 [465 kB]
Fetched 465 kB in 1s (678 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package python3-mpi4p

In [2]:
!sudo apt-get install -y python3-mpi4py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-mpi4py is already the newest version (3.1.3-1build2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [8]:
!pip install line_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 714.8/714.8 kB 7.9 MB/s eta 0:00:00


##Without any parallelization

In [45]:
import math
import cProfile

def calculate_pi(N):
    dx = 1.0 / N
    sum = 0.0
    for i in range(N):
        xi = i * dx
        sum += dx * math.sqrt(1 - xi**2)
    return 4 * sum

N = 10000  # número de subdivisiones

cProfile.run('calculate_pi(N)')

         10004 function calls in 0.007 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.006    0.006    0.007    0.007 <ipython-input-45-5980700521e2>:4(calculate_pi)
        1    0.000    0.000    0.007    0.007 <string>:1(<module>)
        1    0.000    0.000    0.007    0.007 {built-in method builtins.exec}
    10000    0.001    0.000    0.001    0.000 {built-in method math.sqrt}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




##Multiprocessing

In [42]:
import math
from multiprocessing import Pool
import cProfile

def f(i, N):
    dx = 1.0 / N
    xi = i * dx
    return dx * math.sqrt(1 - xi**2)

def parallel_calculate_pi(N):
    with Pool() as pool:
        results = pool.starmap(f, [(i, N) for i in range(N)])
    return 4 * sum(results)

N = 10000

def main():
    print("Aproximation with multiprocessing:", parallel_calculate_pi(N))

cProfile.run('main()')

Aproximation de pi con multiprocessing: 3.1417914776113167
         943 function calls in 0.055 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       12    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:404(parent)
        1    0.000    0.000    0.054    0.054 <ipython-input-42-b12c0a06e735>:10(parallel_calculate_pi)
        1    0.004    0.004    0.004    0.004 <ipython-input-42-b12c0a06e735>:12(<listcomp>)
        1    0.000    0.000    0.055    0.055 <ipython-input-42-b12c0a06e735>:17(main)
        1    0.000    0.000    0.055    0.055 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 __init__.py:219(_acquireLock)
        2    0.000    0.000    0.000    0.000 __init__.py:228(_releaseLock)
        5    0.000    0.000    0.000    0.000 _weakrefset.py:39(_remove)
        5    0.000    0.000    0.000    0.000 _weakrefset.py:86(add)
        6    0.000    0.000    0.000    0.000 connection.py:

##Mpi4py

In [46]:
from mpi4py import MPI
import math
import cProfile

def calculate_part_of_pi(N, rank, size):
    dx = 1.0 / N
    local_sum = sum(dx * math.sqrt(1 - (i * dx) ** 2) for i in range(rank, N, size))
    return local_sum

def distributed_calculate_pi(N):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    profiler = cProfile.Profile()
    profiler.enable()

    local_sum = calculate_part_of_pi(N, rank, size)
    pi_approx = 4 * comm.reduce(local_sum, op=MPI.SUM, root=0)

    profiler.disable()
    profiler.print_stats(sort='time')

    if rank == 0:
        print(f"Aproximación with mpi4py: {pi_approx}")

if __name__ == "__main__":
    N = 10000
    if MPI.COMM_WORLD.Get_rank() == 0:
        print("Starting distributed PI calculation with profiling")
    distributed_calculate_pi(N)

Starting distributed PI calculation with profiling
         20005 function calls in 0.011 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10001    0.008    0.000    0.009    0.000 <ipython-input-46-cc25e2be2c4d>:7(<genexpr>)
        1    0.001    0.001    0.011    0.011 {built-in method builtins.sum}
    10000    0.001    0.000    0.001    0.000 {built-in method math.sqrt}
        1    0.000    0.000    0.000    0.000 {method 'reduce' of 'mpi4py.MPI.Comm' objects}
        1    0.000    0.000    0.011    0.011 <ipython-input-46-cc25e2be2c4d>:5(calculate_part_of_pi)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


Aproximación with mpi4py: 3.1417914776113167
